# Clustering Crypto

In [66]:
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

## Deliverable 1: Preprocessing the Data for PCA

In [67]:
# Load the crypto_data.csv dataset.
file_path = "./Resources/crypto_data.csv"
crypto_data_df = pd.read_csv(file_path, index_col=0)

crypto_data_df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [68]:
# Keep all the cryptocurrencies that are being traded.
crypto_data_df.drop(crypto_data_df[crypto_data_df['IsTrading'] == False].index, inplace=True)

In [69]:
# Remove the "IsTrading" column. 
crypto_data_df_update = crypto_data_df.drop(['IsTrading'], axis=1)
crypto_data_df_update

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [70]:
# Remove rows that have at least 1 null value
crypto_data_df_update_2 = crypto_data_df_update.dropna()
crypto_data_df_update_2

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [71]:
# Keep the rows where coins are mined.
coins_mined_df = crypto_data_df_update_2[crypto_data_df_update_2!=0].dropna()
coins_mined_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [72]:
# Create a new DataFrame that holds only the cryptocurrencies names.
coins_name_df = coins_mined_df['CoinName']
coins_name_df = pd.DataFrame(coins_name_df)
coins_name_df.head()

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [73]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_data_df_update_3 = coins_mined_df.drop(columns='CoinName', axis=1)
crypto_data_df_update_3

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [74]:
# Use get_dummies() to create variables for text features.
X = pd.get_dummies(crypto_data_df_update_3, columns = ['Algorithm', 'ProofType'])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# Standardize the data with StandardScaler().
data_scaler = StandardScaler().fit_transform(X)
data_scaler

array([[-0.11674788, -0.15286468, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.09358885, -0.14499604, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [ 0.52587231,  4.4937636 , -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       ...,
       [-0.09523411, -0.13215444, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11658774, -0.15255408, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ],
       [-0.11674507, -0.15284989, -0.0433555 , ..., -0.0433555 ,
        -0.0433555 , -0.0433555 ]])

## Deliverable 2: Reducing Data Dimensions Using PCA

In [76]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(data_scaler)
crypto_pca

array([[-0.3448706 ,  1.05218953, -0.54780615],
       [-0.32817676,  1.05243908, -0.54818815],
       [ 2.30462569,  1.70956424, -0.65675663],
       ...,
       [ 0.32512818, -2.31426418,  0.40878159],
       [-0.11846267, -2.14467511,  0.37616454],
       [-0.27651772,  0.76877706, -0.29611312]])

In [77]:
# Create a DataFrame with the three principal components.
pcs_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=X.index
)
pcs_df.head()

,PC 1,PC 2,PC 3
42,-0.344871,1.052190,-0.547806
404,-0.328177,1.052439,-0.548188
1337,2.304626,1.709564,-0.656757
BTC,-0.152712,-1.306755,0.176151
ETH,-0.146539,-2.041543,0.380373


## Deliverable 3: Clustering Crytocurrencies Using K-Means
### Finding the Best Value for k Using the Elbow Curve

In [78]:
# Create an elbow curve to find the best value for K.
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range ok k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

C:\Users\snkty\anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:882: UserWarning:

KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.



:Curve   [k]   (inertia)

Running K-Means with k=4

In [79]:
# Initialize the K-Means model.
model = KMeans(n_clusters=4, random_state=4)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Add a new "Class" column
pcs_df["Class"] = model.labels_
pcs_df

,PC 1,PC 2,PC 3,Class
42,-0.344871,1.052190,-0.547806,1
404,-0.328177,1.052439,-0.548188,1
1337,2.304626,1.709564,-0.656757,1
BTC,-0.152712,-1.306755,0.176151,0
ETH,-0.146539,-2.041543,0.380373,0
...,...,...,...,...
ZEPH,2.451086,0.761361,-0.026939,1
GAP,-0.342912,1.052088,-0.547823,1
BDX,0.325128,-2.314264,0.408782,0
ZEN,-0.118463,-2.144675,0.376165,0


In [80]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_data_df_update_3, pcs_df], axis = 1)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df["CoinName"] = coins_name_df["CoinName"]

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df = clustered_df[["Algorithm", 
                             "ProofType",
                             "TotalCoinsMined",
                             "TotalCoinSupply", 
                             "PC 1", 
                             "PC 2", 
                             "PC 3", 
                             "CoinName", 
                             "Class"]]

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(533, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.344871,1.052190,-0.547806,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.328177,1.052439,-0.548188,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.304626,1.709564,-0.656757,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.152712,-1.306755,0.176151,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.146539,-2.041543,0.380373,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168812,-1.125700,0.014693,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.410707,1.278421,-0.495780,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.161697,-2.209141,0.432132,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.144977,-2.041631,0.380361,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.118462,-2.144675,0.376165,ZCash,0


## Deliverable 4: Visualizing Cryptocurrencies Results
### 3D-Scatter with Clusters

In [81]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x = "PC 1",
    y = "PC 2",
    z = "PC 3",
    hover_name = "CoinName",
    hover_data = ["Algorithm"],
    color = "Class",
    symbol = "Class",
    width = 800
)
fig.update_layout(legend = dict(x = 0, y = 1))
fig.show()

In [82]:
# Create a table with tradable cryptocurrencies.
tradable_crypto = clustered_df.hvplot.table(columns=['CoinName', 
                                                           'Algorithm', 
                                                           'ProofType',
                                                           'TotalCoinSupply', 
                                                           'TotalCoinsMined', 
                                                           'Class'], sortable=True, selectable=True)
tradable_crypto

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [83]:
# Print the total number of tradable cryptocurrencies.
print(f'There are {len(clustered_df)} tradable cryptocurrencies.')

There are 533 tradable cryptocurrencies.


In [84]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_cluster = clustered_df[['TotalCoinSupply', 'TotalCoinsMined']].copy()
X_cluster_scaled = MinMaxScaler().fit_transform(X_cluster)
X_cluster_scaled

array([[4.20000000e-11, 5.94230127e-03],
       [5.32000000e-04, 7.00182308e-03],
       [3.14159265e-01, 3.53420682e-02],
       ...,
       [1.40022261e-03, 6.92655266e-03],
       [2.10000000e-05, 5.94962775e-03],
       [1.00000000e-06, 5.94243008e-03]])

In [85]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
plot_df = pd.DataFrame(X_cluster_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)

# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
plot_df['CoinName'] = clustered_df.CoinName

# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df['Class'] = clustered_df.Class

plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,4.200000e-11,0.005942,42 Coin,1
404,5.320000e-04,0.007002,404Coin,1
1337,3.141593e-01,0.035342,EliteCoin,1
BTC,2.100000e-05,0.005960,Bitcoin,0
ETH,0.000000e+00,0.006050,Ethereum,0
LTC,8.400000e-05,0.006006,Litecoin,0
DASH,2.200000e-05,0.005951,Dash,1
XMR,0.000000e+00,0.005960,Monero,0
ETC,2.100000e-04,0.006056,Ethereum Classic,0
ZEC,2.100000e-05,0.005950,ZCash,0


In [86]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(x='TotalCoinsMined', y='TotalCoinSupply', hover_cols='CoinName', by='Class')

:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)